# Import Dependencies

In [28]:
import cv2
import os
import logging
import time
import numpy as np
import mediapipe as mp
import json
from tqdm.notebook import tqdm # for progress bars in Jupyter Notebook
from matplotlib import pyplot as plt

## Detecting Keypoints using MP Holistic

In [29]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [30]:
def setup_holistic_detection():
    # Initialize Camera Input
    cap = cv2.VideoCapture(0)

    # Set Up MediaPipe Holistic Model
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    return cap, holistic

In [31]:
def process_mp_frames(frame, holistic):
    # Opencv records in BGR while mediapipe supports RGB
    # We need to recolor frame to RGB to support MediaPipe processing
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  # Make Image non-writeable for performance
    results = holistic.process(image)              # process image and return object contain landmarks
    image.flags.writeable = True                   # Back to writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color back to BGR for OpenCV 

    return image, results

In [32]:
def draw_landmarks(image, results):
    # Draw face landmarks
    mp_drawing.draw_landmarks(image, 
                              results.face_landmarks, 
                              mp_holistic.FACEMESH_TESSELATION, # FACEMESH_CONTOURS could also be valid here
                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=(170,86,0), thickness=1, circle_radius=2),
                              connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
    # Draw pose connections
    mp_drawing.draw_landmarks(image, 
                              results.pose_landmarks, 
                              mp_holistic.POSE_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, 
                              results.left_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(), 
                              connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style())
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, 
                              results.right_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(), 
                              connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style())
    

In [33]:
def extract_keypoints_comprehensive(results):
    # Pose Landmarks have 33 Keypoints
    # if results.pose_landmarks:
    #     pose_landmarks = []
    #     for res in results.pose_landmakrs.landmark:
    #         pose_landmarks.extend([res.x, res.y, res.z, res.visibility])
    #     pose = np.array(pose_landmarks)
    # else:
    #     pose = np.zeros(33*4)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    keypoints_vector = np.concatenate([pose, face, lh, rh])
    # print(keypoints_vector)
    return keypoints_vector

In [34]:
def run_holistic_detection():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow loggin
    logging.getLogger('mediapipe').setLevel(logging.ERROR)  # Only show errors from mediapipe
    cap, holistic = setup_holistic_detection()

    while cap.isOpened():
        # Read frame
        ret, frame = cap.read()
        if not ret:
            print('camera stopped: no frames grabbed')
            break

        # Process Frame/Make dectections
        image, results = process_mp_frames(frame, holistic)
        # print(results)

        # Draw Landmarks
        draw_landmarks(image, results)

        # Extract Keypoints
        keypoints_vector = extract_keypoints_comprehensive(results)

        # Show to screen
        cv2.imshow('ASL Detection', image)

        # Exit on 'q' press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            draw_landmarks(frame, results)
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            break

    cap.release()
    cv2.destroyAllWindows()

In [35]:
#run_holistic_detection()

## Dataset Collection Pipeline for Pre-built Dataset

In [36]:
WLASL_PATH = "WLASL_DATA"
VIDEOS_PATH = os.path.join(WLASL_PATH, "videos")
JSON_PATH = os.path.join(WLASL_PATH, "WLASL_v0.3.json")
OUTPUT_PATH = "WLASL_Processed"

# Create Output directory
os.makedirs(OUTPUT_PATH, exist_ok=True)

**Verify Dataset Structure**

In [37]:
print("WLASL folder contents:")
print(os.listdir(WLASL_PATH))

# If there's a JSON file, check its structure
json_path = os.path.join(WLASL_PATH, "WLASL_v0.3.json")
if os.path.exists(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    print(f"\nJSON file contains data for {len(data)} signs")
    print(f"Example of first sign: {data[0]['gloss']}")

WLASL folder contents:
['nslt_2000.json', 'nslt_100.json', '.DS_Store', 'nslt_300.json', 'WLASL_v0.3.json', 'nslt_1000.json', 'wlasl_class_list.txt', 'videos', 'missing.txt']

JSON file contains data for 2000 signs
Example of first sign: book


**Process Videos**

In [38]:
def process_video(video_path, output_dir, max_frames=30):
    """Process a single video file and extract landmarks"""
    # Create Output Directory
    os.makedirs(output_dir, exist_ok=True)

    # open video
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling rate
    if frame_count <= max_frames:
        # Use all frames is video is short
        frame_indices = list(range(frame_count))
    else:
        # Sample frames evenly if video is long
        frame_indices = np.linspace(0, frame_count-1, max_frames, dtype=int)

    # Initialize MediaPipe
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Process Frames
        for frame_idx, i in enumerate(frame_indices):
            # Set frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            success, frame = cap.read()

            if not success:
                print(f"Could not read frame {i} from {video_path}")
                continue
                
            # Process image
            image, results = process_mp_frames(frame, holistic)
            
            # Extract and savve keypoints
            keypoints_vector = extract_keypoints_comprehensive(results)
            npy_path = os.path.join(output_dir, str(frame_idx))
            np.save(npy_path, keypoints_vector)

    cap.release()
    return len(frame_indices)

In [39]:
def process_wlasl_dataset(max_signs=None, max_videos_per_sign=30):
    """Process the WLASL dataset"""
    # Load WLASL metadata
    with open(JSON_PATH, 'r') as f:
        wlasl_data = json.load(f)

    # Limit number of signs if specified
    if max_signs:
        wlasl_data = wlasl_data[:max_signs]

    # list to store sign info
    processed_signs = []

    # Create sign to ID mapping
    sign_map = {sign_data['gloss']: idx for idx, sign_data in enumerate(wlasl_data)}

    # Save sign map for later use
    with open(os.path.join(OUTPUT_PATH, 'sign_map.json'), 'w') as f:
        json.dump(sign_map, f, indent=2)

    # Process each sign
    for sign_idx, sign_data in enumerate(tqdm(wlasl_data, desc="Processing signs")):
        sign_name = sign_data['gloss']
        sign_id = sign_idx  # Use index as class ID
        
        # Create directory for this sign
        sign_dir = os.path.join(OUTPUT_PATH, sign_name)
        os.makedirs(sign_dir, exist_ok=True)

        # Process videos for this sign
        videos_processed = 0
        
        for i, instance in enumerate(sign_data['instances']):
            if videos_processed >= max_videos_per_sign:
                break
                
            video_id = instance['video_id']
            video_path = os.path.join(VIDEOS_PATH, f"{video_id}.mp4")
            
            # Check if video exists
            if not os.path.exists(video_path):
                print(f"Video not found: {video_path}")
                continue
            
            # Process video
            output_dir = os.path.join(sign_dir, str(videos_processed))
            frames_processed = process_video(video_path, output_dir)
            
            if frames_processed > 0:
                videos_processed += 1
                print(f"Processed {sign_name} video {videos_processed}/{max_videos_per_sign}")
        
        # Add to processed signs list
        processed_signs.append({
            'name': sign_name,
            'id': sign_id,
            'videos_processed': videos_processed
        })
        
        print(f"Completed sign: {sign_name} - {videos_processed} videos processed")
    
    # Save processed sign information
    with open(os.path.join(OUTPUT_PATH, 'processed_signs.json'), 'w') as f:
        json.dump(processed_signs, f, indent=2)
    
    return processed_signs

In [ ]:
# Testing with small subset
process_wlasl_dataset(max_signs=10, max_videos_per_sign=10)

Processing signs:   0%|          | 0/10 [00:00<?, ?it/s]

I0000 00:00:1743457790.985194  142202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1743457791.046814  160996 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457791.061064  160995 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457791.064439  160994 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457791.064524  160999 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457791.064538  161003 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

Processed book video 1/10
Video not found: WLASL_DATA/videos/65225.mp4
Video not found: WLASL_DATA/videos/68011.mp4
Video not found: WLASL_DATA/videos/68208.mp4
Video not found: WLASL_DATA/videos/68012.mp4
Video not found: WLASL_DATA/videos/70212.mp4
Video not found: WLASL_DATA/videos/70266.mp4
Video not found: WLASL_DATA/videos/07085.mp4
Video not found: WLASL_DATA/videos/07086.mp4
Video not found: WLASL_DATA/videos/07087.mp4


I0000 00:00:1743457792.932713  142202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1743457792.990474  161149 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457793.003061  161154 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457793.004443  161156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457793.004483  161152 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457793.004497  161153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

Processed book video 2/10
Video not found: WLASL_DATA/videos/07088.mp4
Video not found: WLASL_DATA/videos/07089.mp4
Video not found: WLASL_DATA/videos/07090.mp4
Video not found: WLASL_DATA/videos/07091.mp4
Video not found: WLASL_DATA/videos/07092.mp4
Video not found: WLASL_DATA/videos/07093.mp4


...

I0000 00:00:1743457854.294454  142202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1743457854.356448  163864 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457854.369625  163871 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457854.371087  163864 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457854.371094  163866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457854.372022  163870 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

Processed chair video 5/10


I0000 00:00:1743457855.756435  142202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1743457855.818791  163895 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457855.833041  163895 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457855.834263  163896 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457855.834350  163899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457855.835094  163894 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

Processed chair video 6/10
Video not found: WLASL_DATA/videos/09853.mp4


I0000 00:00:1743457857.124050  142202 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1743457857.184880  163968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457857.199293  163973 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457857.200649  163969 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457857.200744  163968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743457857.201343  163975 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

In [ ]:
# Process Entire Dataset
process_wlasl_dataset()

## Dataset Colelction Pipeline for Custom Camera Recordings

In [9]:
# Create Directories to store data
def create_directories(actions):
    DATA_PATH = os.path.join('MP_Data')

    # Detecting Actions
    for action in actions:
        # 30 is often considered minimum threshold to detect meaningful patterns
        # n >= 30 commonly used as rule of thumb
        for sequence in range(30): # 30 videos per action 
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass

def collect_asl_data():
    # Define Actions - starting with a small set for testing
    actions = np.array(['hello', 'thanks', 'iloveyou', 'yes', 'no', 'please', 'sorry'])

    # create directories for data collection - 30 directories per action
    create_directories(actions)

    # Define data path
    DATA_PATH = os.path.join('MP_Data')

    # set up holistic model
    cap, holistic = setup_holistic_detection()

    # Loop through different actions
    for action in actions:
        # loop through sequences
        for sequence in range(30):
            # loop through video length of 30 frames
            for frame_num in range(30):
                # Read Feed
                ret, frame = cap.read()
                if not ret:
                    print('Failed to grab frame')
                    break

                # Process the frame for landmark detection
                image, results = process_mp_frames(frame, holistic)

                # Draw Landmarks on the image
                draw_landmarks(image, results)

                # Apply status text to the frame
                if frame_num == 0:
                    # Visual Countdown before starting each sequence
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting for {action} - Sequence {sequence}', (15, 15), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.imshow('ASL Data Collection', image)
                    cv2.waitKey(2000)  # Wait 2 seconds before starting
                else:
                    # Display collection status
                    cv2.putText(image, f'Collecting for {action} - Sequence {sequence}', (15, 15), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.putText(image, f'Frame {frame_num}', (15, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # show to screen
                cv2.imshow('ASK Data Collection', image)

                # Export keypoints
                keypoints_vector = extract_keypoints_comprehensive(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints_vector)
                
                # Break gracefully on 'q' press
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return
            
            # Short break between sequences
            if sequence < 29:  # Don't show after the last sequence
                # Display break message
                ret, frame = cap.read()
                if ret:
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    cv2.putText(image, 'SEQUENCE COMPLETE', (120, 200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Preparing for next sequence...', (120, 230), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.imshow('ASL Data Collection', image)
                cv2.waitKey(2000)  # 2 second break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Data collection complete!")

In [10]:
collect_asl_data()

2025-03-28 11:00:29.235 Python[40342:16951070] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1743174030.624805 16951070 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743174030.698996 16952277 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743174030.714151 16952270 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743174030.716951 16952272 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743174030.717010 16952276 infe

[0. 0. 0. ... 0. 0. 0.]
[ 0.52983934  0.65255779 -0.81979042 ...  0.27065799  0.56299394
 -0.03344227]
[ 0.53177369  0.65267652 -0.83753949 ...  0.28021303  0.55900109
 -0.02965582]


W0000 00:00:1743174032.920365 16952272 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[ 0.53282154  0.65288162 -0.73916668 ...  0.28228977  0.56940174
 -0.03039847]
[ 0.53346562  0.65289038 -0.75328124 ...  0.28090003  0.58425015
 -0.03425811]
[ 0.5342502   0.65591425 -0.70599139 ...  0.27849773  0.59796286
 -0.0335652 ]
[ 0.53548741  0.65877414 -0.73717904 ...  0.27847347  0.60502571
 -0.03121363]
[ 0.53625637  0.66370994 -0.74308914 ...  0.27984151  0.6112386
 -0.03471507]
[ 0.53715897  0.66643572 -0.7489261  ...  0.28361732  0.61474425
 -0.03286409]
[ 0.53760546  0.66778022 -0.66063136 ...  0.28296655  0.61062664
 -0.02941914]
[ 0.53758371  0.66822898 -0.64128596 ...  0.28318027  0.60386109
 -0.02978708]
[ 0.53777122  0.6710366  -0.65361899 ...  0.28726658  0.58771473
 -0.02845259]
[ 0.53750038  0.66830295 -0.67829019 ...  0.28796375  0.55751872
 -0.02893239]
[ 0.53702503  0.66762495 -0.77733433 ...  0.28608632  0.53952688
 -0.02583328]
[ 0.5369252   0.66344428 -0.76855367 ...  0.28514442  0.52349126
 -0.0285382 ]
[ 0.5356127   0.65518117 -0.86727172 ...  0.28295818 